看了前面几个视频，给出的notebook都不是很好，都不能运行，终于这次视频给出的代码是可运行的了，本文主要是介绍怎么使用tf来对文章做summary，很cool的功能，也很实用，想想我自己读完一篇文章后都不能做总结，现在机器可以做了，真是令人兴奋

本文使用的数据集是：https://metamind.io/research/the-wikitext-long-term-dependency-language-modeling-dataset/

这些数据都是从Wikipedia的文章上解析出来的100million的tokens

In [ ]:
# dependencies

import numpy as np #vectorization
import random #generate probability distribution 
import tensorflow as tf #ml
import datetime #clock training time

下一步我们将读取文章内容,数据的下载地址是：https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip

In [ ]:
text = open('wiki.test.raw').read()
print('text length in number of characters:', len(text))

print('head of text:')
print(text[:100]) #all tokenized words, stored in a list called text

下一步我们要将数据进行排序，看有多少的字符

In [ ]:
chars = sorted(list(set(text)))
char_size = len(chars)
print('number of characters:', char_size)
print(chars)

下一步我们要建立两个字典，一个是index => character，另一个是character => index

In [ ]:
char2id = dict((c, i) for i, c in enumerate(chars))
id2char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
def sample(prediction):
    #Samples are uniformly distributed over the half-open interval 
    r = random.uniform(0,1)
    #store prediction char
    s = 0
    #since length > indices starting at 0
    char_id = len(prediction) - 1
    #for each char prediction probabilty
    for i in range(len(prediction)):
        #assign it to S
        s += prediction[i]
        #check if probability greater than our randomly generated one
        if s >= r:
            #if it is, thats the likely next char
            char_id = i
            break
    #dont try to rank, just differentiate
    #initialize the vector
    char_one_hot = np.zeros(shape=[char_size])
    #that characters ID encoded
    #https://image.slidesharecdn.com/latin-150313140222-conversion-gate01/95/representation-learning-of-vectors-of-words-and-phrases-5-638.jpg?cb=1426255492
    char_one_hot[char_id] = 1.0
    return char_one_hot

接着我们要做的是怎么将输入转换为向量

在lstm中都会有个连续输入的值，然后导出一个输出，形象点就是：
![](http://karpathy.github.io/assets/rnn/diags.jpeg)

在[The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)文章中有介绍具体的模型，我们有多个输入来预测一个输出

In [ ]:
len_per_section = 50
skip = 2
sections = []
next_chars = []
#fill sections list with chunks of text, every 2 characters create a new 50 
#character long section
#because we are generating it at a character level
for i in range(0, len(text) - len_per_section, skip):
    sections.append(text[i: i + len_per_section])
    next_chars.append(text[i + len_per_section])

In [ ]:
print(sections[0],"\nPredictions:",next_chars[0])

In [ ]:
print(sections[1],"\nPredictions:",next_chars[1])

我们可以看到sections[1]比sections[0]就是多移动了skip=2位，然后每len_per_section=50预测1个character

下一步我们就是将输入和输出转换为one-hot encoded的形式：

In [ ]:
X = np.zeros((len(sections), len_per_section, char_size))
#label column for all the character id's, still zero
y = np.zeros((len(sections), char_size))
#for each char in each section, convert each char to an ID
#for each section convert the labels to ids 
for i, section in enumerate(sections):
    for j, char in enumerate(section):
        X[i, j, char2id[char]] = 1
    y[i, char2id[next_chars[i]]] = 1